## 隐式详解
摘抄自《scala程序设计（第2版）》————第 5 章 隐式详解

隐式(implicit)是 Scala 的一个强大的特性,同时也是一个可能存在争议的特性。使用隐式能够减少代码,能够向已有类型中注入新的方法,也能够创建领域特定语言(DSL)。

隐式之所以会产生争议,是因为除了通过 Predef 对象自动加载的那些隐式对象外,其他在源码中出现的隐式对象均不是本地对象。隐式对象一旦进入作用域,编译器便能执行该隐式对象以生成方法参数或将指定参数转化成预期类型。不过在阅读源代码时,读者无法简单地指出什么时候会应用这些隐式值和隐式方法,而这可能会给该读者造成困惑。幸运的是随着经验的累积,你能够意识到什么时候将会触发这些隐式对象,你也可以通过阅读这些对象的 API 来学习这些知识。不过对于初学者而言,这将是一段意外之旅。

想要理解隐式对象的工作机制需要采用较为直接的学习方法。本章的大多数篇幅将通过示例讲解隐式对象能够解决的问题。

### 隐式参数
在 2.5.3 节中,我们使用了 implicit 关键字标记那些用户无需显式提供的方法参数。调用方法时,如果未输入隐式参数且代码所处作用域中存在类型兼容值时,类型兼容值会从作用域中调出并被使用,反之,系统将会抛出编译器错误。

假设我们定义了一个用于计算销售税的方法,而税率被设置为隐式参数。
```scala
//计算税(calculate:计算;核算, tax:税)
def calcTax(amount: Float)(implicit rate: Float): Float = amount * rate
```

调用该方法时,系统会将代码所在局部作用域中的某一隐式值传入此方法:
```scala
implicit val currentTaxRate = 0.08F
...
val tax = calcTax(50000F) // 4000.0
```

对于某些简单的场景,设定一个固定的浮点数就能满足需求。不过有些场景可能就不这么简单了。例如某些应用需要知道当前事务发生的具体地点,以便增收地方税。而为了促进购物消费,某些辖区也可能会将年假的最后几天设定为“免税期”。

幸运的是,我们可以使用隐式方法解决这一问题。隐式对象本身不具有任何参数,除非该参数同样被标示为隐式参数。下面列举了计算销售税的完整示例:
```scala
// src/main/scala/progscala2/implicits/implicit-args.sc

// Never use Floats for money:
// 永远不要用FLoat类型表示货币:
def calcTax(amount: Float)(implicit rate: Float): Float = amount * rate

object SimpleStateSalesTax {
  implicit val rate: Float = 0.05F
}

case class ComplicatedSalesTaxData(
  baseRate: Float,
  isTaxHoliday: Boolean,
  storeId: Int)

object ComplicatedSalesTax {
  private def extraTaxRateForStore(id: Int): Float = {
    // From id, determine location, then extra taxes...
    // 可以通过id推断出商铺所在地,之后再计算附加税......
    0.0F
  }

  implicit def rate(implicit cstd: ComplicatedSalesTaxData): Float = 
    if (cstd.isTaxHoliday) 0.0F
    else cstd.baseRate + extraTaxRateForStore(cstd.storeId)
}

{
  import SimpleStateSalesTax.rate

  val amount = 100F
  //使用  implicit val rate: Float = 0.05F  作为参数
  println(s"Tax on $amount = ${calcTax(amount)}")
}

{
  import ComplicatedSalesTax.rate
  implicit val myStore = ComplicatedSalesTaxData(0.06F, false, 1010)

  val amount = 100F
  //使用  ComplicatedSalesTax.rate(myStore)的转换结果  作为参数（同时使用了scala的两种隐式：隐式值和隐式转换函数）
  println(s"Tax on $amount = ${calcTax(amount)}")
}
```

尽管我们是在可插入字符串(interpolated string)中调用 calcTax 方法,但该方法仍然会将隐式值应用到 rate 参数上。

还有一类更复杂的情景:我们可以定义一个包含了隐式参数的隐式方法,该隐式参数将接收方法所需的数据。

运行该脚本将得到以下输出:
```
Tax on 100.0 = 5.0
Tax on 100.0 = 6.0
```

**调用 implicitly 方法:**

Predef 对象中定义了一个名为 implicitly 的方法。如果将 implicitly 方法与附加类型签名(type signature addition)相结合,便能以一种有用且快捷的方式定义一个接收参数化类型隐式参数的函数。

在下列示例中,我们使用了这种方法对 List 的 sortBy 方法进行封装。
```scala
// src/main/scala/progscala2/implicits/implicitly-args.sc
import math.Ordering

case class MyList[A](list: List[A]) {
  def sortBy1[B](f: A => B)(implicit ord: Ordering[B]): List[A] =
    list.sortBy(f)(ord)

  def sortBy2[B : Ordering](f: A => B): List[A] =
    list.sortBy(f)(implicitly[Ordering[B]])
}

val list = MyList(List(1,3,5,2,4))

list sortBy1 (i => -i)
list sortBy2 (i => -i)
```
有些集合提供了一些排序方法, List.sortBy 便是其中之一。 List.sortBy 方法的第一个参数类型为函数,该输入函数能够将函数的输入参数转化为另一个满足 math.Ordering 条件的 类 型。 而 math.Ordering 是 与 Java 中 的 Comparable 抽象等同的类型。 List.sortBy 方法的另一个参数则为隐式参数,该参数知道如何对类型 B 的实例进行排序。

MyList 类提供了两种方式编写像 sortBy 类型的方法。第一种实现: sortBy1 方法应用了我们已知的语法。该方法接受一个额外的类型为 Ordering[B] 的隐式值作为其输入。调用sortBy1 方法时,在当前作用域中一定存在某一 Ordering[B] 的对象实例,该实例清楚地知道如何对我们所需要的 B 类型对象进行排序。我们认为 B 的界限被“上下文”所限定,在这个例子中,上下文限定了 B 对实例进行排序的能力。

**由于这种 Scala 方言应用非常普遍,因此 Scala 提供了一个简化版的语法,这正是第二类实现 sortBy2 所使用的语法。类型参数 B : Ordering 被称为 上下文定界 (context bound), 它暗指第二个参数列表(也就是那个隐式参数列表)将接受 Ordering[B] 实例。**


**不过,我们仍需要在方法中访问 Ordering 对象实例。由于在源代码中我们不再明确地声明 Ordering 对象实例,因此这个实例没有自己的名称。针对这种现象我们该怎么办呢?Predef.implicitly 方法帮我们解决了这个问题。** implicitly 方法会对传给函数的所有标记为隐式参数的实例进行解析。请注意 implicitly 方法所需要的类型签名,在本例中Ordering[B] 是其类型签名。

当我们需要类型为参数化类型的隐式参数时,当类型参数属于当前作用域的其他一些类型时(例如:[B : Ordering] 代表了类型为 Ordering[B] 的隐式参数),可以将上下文定界(context bound)与 implicitly 方法结合起来,以简洁地方式解决这个问题。

### 隐式参数适用的场景
程序员应谨慎明智地使用隐式对象。滥用隐式对象会让读者无法理解代码的用意。

既然隐式参数具有弊端,那么我们为什么还要使用它呢?目前已经有不少通过隐式参数实现的常见习语(idiom),隐式参数为这些习语带来两类好处:第一类好处是能够消除样板代码,例如隐式对象提供了上下文信息以省略明确指定这些信息的代码;第二类好处是通过引入约束来减少 bug 数量以及使用参数化类型对某些方法允许的输入参数类型进行限定。下面我们将对这些习语进行分析。

#### 执行上下文
在 2.5.3 节,我们学习了关于 Future 对象的示例,示例中的 apply 方法的第二个参数列表被设为隐式参数,该参数用于将 ExecutionContext 对象传递给 Future.apply 方 法:
```scala
apply[T](body: => T)(implicit executor: ExecutionContext): Future[T]
```
一些其他的方法同样提供了这类隐式参数。

尽管我们在调用这些方法时并未指定 ExecutionContex t 对象,但是我们导入了可供编译器使用的全局默认值:
```scala
import scala.concurrent.ExecutionContext.Implicits.global
```

**使用隐式参数传入“执行上下文”是一个值得推荐的用法。另外,编写事务、数据库连接、线程池以及用户会话时隐式参数上下文也同样适合使用。使用方法参数能组合行为,而将方法参数设置为隐式参数能够使 API 变得更加简洁。**

#### 功能控制
除了能够传递上下文对象之外,隐式参数还具有控制系统功能的作用。

举个例子,通过引入授权令牌,我们可以控制某些特定的 API 操作只能供某些用户调用,我们也可以使用授权令牌决定数据可见性,而隐式用户会话参数也许就包含了这类令牌信息。

假设你想要创建用户界面的菜单,其中某些菜单项只对已登录用户可见,而其他菜单项则仅对未登录用户可见。
```scala
def createMenu(implicit session: Session): Menu = {
  val defaultItems = List(helpItem, searchItem)
  val accountItems =
    if (session.loggedin()) List(viewAccountItem, editAccountItem)
    else List(loginItem)
  Menu(defaultItems ++ accountItems)
}
```

#### 限定可用实例
设想一下,我们希望对具有参数化类型方法中的类型参数进行限定,使该参数只接受某些类型的输入。那么该如何处理呢?

假如允许输入的所有参数类型均为某一公共超类的子类型,那么无需应用隐式技术,面向对象的技术便可解决这一问题。让我们首先思考一下解决方案。

在 3.10 节中,我们通过示例演示掌握了如何实现资源管理器:
```scala
object manage {
  def apply[R <: { def close():Unit }, T](resource: => R)(f: R => T) = {...}
  ...
}
```

类型参数 R 必须是定义了 close():Unit 方法的任意类型的子类。否则,我们管理的所有资源都必须实现 Closable 的 trait(请回顾 Scala 是如何使用 trait 取代并扩展 Java 接口的。详情参见 3.14 节):
```scala
trait Closable {
  def close(): Unit
}
...
object manage {
  def apply[R <: Closable, T](resource: => R)(f: R => T) = {...}
  ...
}
```
假如这些类型并无公共超类,这项技术便无用武之地。对于这种情况,我们可以使用隐式参数对允许的类型进行限定。Scala 集合 API 就是利用这项技术解决了一些设计上的问题。

具 体 的 集 合 类 所 支 持 的 某 些 方 法 是 由 父 类 实 现 的。 例 如: List[A].map(f: A => B):List[B] 方法首先将函数 f 运用在各个元素之上,之后又创建了一个新的列表。 由于大多数的集合都提供了 map 方法,因此在一个通用的 trait 中实现 map 方法,再将该 trait 混入(我们在 3.14 节中讨论了如何使用 trait 实现“混入”)到需要该方法的集合中也就顺理成章了。不过假如我们希望 map 方法能返回与容器元素类型相同的类型,那么又该使用什么机制通知 map 方法呢?

Scala API 运用一种常见的手法,将一个“构建器”(builder)作为隐式参数传入到 map 方法中。该构建器知道如何构造一个同种类型的新容器。这实际上与定义在 TraversableLike 中的 map 方 法 的 签 名 很 相 似。 TraversableLike 是 一 个 trait, 它 被 混 入 了 那 些“ 可 遍 历 ”(traversable)的容器类型中。
```scala
trait TraversableLike[+A, +Repr] extends ... {
  ...
  def map[B, That](f: A => B)(
    implicit bf: CanBuildFrom[Repr, B, That]): That = {...}
  ...
}
```
我们再回顾一下, +A 意味着 TraversableLike[A] 类型是 A 类型的协变(covariant);假如 B类型是 A 类型的子类型,那么 TraversableLike[B] 类型也是 TraversableLike[A] 类型的子类型。

CanBuildFrom 便是我们所使用的构造器。只要存在一个隐式构建器对象,你便能够构建出任意一个你想要的新容器。为了强调这点,该构建器被命名为 CanBuildForm 。

实际上,容器通过 Repr 类型持有内部的元素,而 B 类型则是函数 f 所生成的元素的类型。B 便是我们想要创建的目标集合的类型参数。通常情况下,我们希望构建一个与输入类型相同新的容器,允许类型参数有所差异。也就是说, B 也许与 A 类型相同,也许不同。Scala API 为所有内置的容器类型提供了隐式的 CanBuildFrom 构造器。

因此, map 操作可以输出的集合类型是由当前存在的对应的 CanBuildFrom 构造器实例所决定的,而这些构造器在当前作用域被声明为隐式对象。假如你自定义了某些容器,而你希望能够复用像 TraversableLike.map 这样的方法实现,你需要创建 CanBuildFrom 类型,并在这些容器的代码中导入它们的隐式示例。

下面我们将学习另一个示例:你希望编写 Java 数据库 API 的 Scala 封装类。这个示例受到了 Cassandra 数据库 API(https://github.com/datastax/java-driver)的启发:
```scala
// src/main/scala/progscala2/implicits/java-database-api.scala

// A Java-like Database API, written in Scala for convenience.
// 为了方便用户使用,我们使用Scala编写了数据库API,该API与Java API较为类似。
package progscala2.implicits {
  package database_api {

    case class InvalidColumnName(name: String)
      extends RuntimeException(s"Invalid column name $name")

    trait Row {
      def getInt   (colName: String): Int
      def getDouble(colName: String): Double
      def getText  (colName: String): String
    }
  }

  package javadb {
    import database_api._

    case class JRow(representation: Map[String,Any]) extends Row {
      private def get(colName: String): Any = 
        representation.getOrElse(colName, throw InvalidColumnName(colName))

      def getInt   (colName: String): Int    = get(colName).asInstanceOf[Int]
      def getDouble(colName: String): Double = get(colName).asInstanceOf[Double]
      def getText  (colName: String): String = get(colName).asInstanceOf[String]
    }

    object JRow {
      def apply(pairs: (String,Any)*) = new JRow(Map(pairs :_*))
    }
  } 
}
```
为了方便起见,我使用 Scala 语言编写了这个 API。API 使用 Map 表示结果集中的一行,不过考虑到效率问题,在现实的实现中也许会使用字节数组表示一行数据。

getInt 、 getDouble 、 getText 以及其他一些尚未实现的一组方法便构成了该 API 的核心功能。这些方法将某一列的原始数据转化为具有正确类型的值,假如你对某一列使用了错误的类型方法,这些方法将抛出 ClassCaseException 异常。

如果我们只定义一个 get[T] 方法,其中 T 代表某一允许的列值类型,会不会更好一些呢?这有助于提供更加统一的调用接口,因为调用这一方法时我们不再需要 case 语句选择正确的调用方法,而且有时候我们还能在这些接口中使用类型推导。

在 Java 中,原始类型与引用类型的区别之一在于我们无法在像 get[T] 这样的参数化方法中使用原始类型。我们必须使用装箱后的类型,比如使用 java.lang.Integer 类型来替代int 类型。但是在高性能数据应用程序中我们往往不希望出现装箱操作的性能损耗!

不过,我们在 Scala 中可以这样做:
```scala
// src/main/scala/progscala2/implicits/scala-database-api.scala

// A Scala wrapper for the Java-like Database API.
// 运用Scala封装对象实现类Java的数据库API。
package progscala2.implicits {
    package scaladb {
    object implicits {
      import javadb.JRow

      implicit class SRow(jrow: JRow) {
        def get[T](colName: String)(implicit toT: (JRow,String) => T): T =
          toT(jrow, colName)
      }

      implicit val jrowToInt: (JRow,String) => Int =
        (jrow: JRow, colName: String) => jrow.getInt(colName)
      implicit val jrowToDouble: (JRow,String) => Double =
        (jrow: JRow, colName: String) => jrow.getDouble(colName)
      implicit val jrowToString: (JRow,String) => String =
        (jrow: JRow, colName: String) => jrow.getText(colName)
    }

    object DB {
      import implicits._

      def main(args: Array[String]): Unit = {
        val row = javadb.JRow("one" -> 1, "two" -> 2.2, "three" -> "THREE!")

        val oneValue1: Int      = row.get("one")
        val twoValue1: Double   = row.get("two")
        val threeValue1: String = row.get("three")
        // val fourValue1: Byte    = row.get("four")  // won't compile  // 不编译该行

        println(s"one1   -> $oneValue1")
        println(s"two1   -> $twoValue1")
        println(s"three1 -> $threeValue1")

        val oneValue2   = row.get[Int]("one")
        val twoValue2   = row.get[Double]("two")
        val threeValue2 = row.get[String]("three")
        // val fourValue2    = row.get[Byte]("four")  // won't compile  // 不编译该行

        println(s"one2   -> $oneValue2")
        println(s"two2   -> $twoValue2")
        println(s"three2 -> $threeValue2")
      }
    }
  }
}
```

在隐式对象中,我们使用了一个隐式类对 Java 的 JRow 类进行封装,而该封装类中提供了我们想要的 get[T] 方法。我们将这些类称为隐式转换(implicit conversion)。这一知识点我们将在本章后面的篇幅中谈论到。现在,你只需要了解使用隐式转换后我们可以对 JRow实例调用 get[T] 方法,就好像该方法就是为该实例定义的那样。

get[T] 方法接受两个参数列表,第一个参数列表是从行中读取数据所需要使用到的列名,第二个是一个隐式函数参数。该函数将抽取行中某一列的数据,并将该列数据转化成正确的类型。

假如你仔细阅读 get[T] 方法,你会注意到该方法引用了 jrow 实例,而 jrow 实例被传递给了 SRow 的构造方法。不过,我们并未使用 val 关键字声明,因此 jrow 并不是该类的成员。那么 get[T] 方法是如何引用这个值的呢?很简单,由于 jrow 位于类体作用域内, get[T]可以直接使用它。

**有时某些构造参数并未被声明为一个字段(使用 val 或 var),这是因为这些参数并未持有类型的状态信息,因此也无需暴露给客户。不过由于这些参数位于整个类型体的作用域内,类型的其他成员仍可以引用它们。**

我们接下来定义了三个函数类型的隐式值,这些函数输入 JRow ,返回具有正确类型的列值。为了能够调用 get[T] ,这些函数使用了 implicitly 方法。

最后,我们定义用于测试的 DB 对象。该对象首先创建了一个 JRow 对象,之后对 JRow 对象调用 get[T] 方法,获得了三列的数值。 DB 对象执行了两遍这样的操作。第一遍执行操作时,系统能够通过变量类型推导出 T 类型,例如:根据 oneValue1 的类型推导出 T 类型。而第二次执行操作时,我们省略了变量类型注释,明确地指定 get[T] 中 T 对应的参数值。因此,我更青睐于第二种方式。

如 果 想 要 执 行 该 代 码, 我 们 需 要 启 动 sbt 并 输 入 run-main progscala2.implicits.scaladb.DB 命令。该命令将会按需编译代码:
```scala
> run-main progscala2.implicits.scaladb.DB
[info] Running scaladb.DB
one1 -> 1
two1 -> 2.2
three1 -> THREE!
one2 -> 1
two2 -> 2.2
three2 -> THREE!
[success] Total time: 0 s, ...
```
请注意源代码中注释了抽取字节值对应的代码行。假如移除了这些行中的 // 字符,编译将会出现错误。下面列出了在移除第一行注释代码后的错误信息(为了适应图书大小,我们在错误信息中添加了换行符)。
```scala
[error] .../implicits/scala-database-api.scala:31: ambiguous implicit values:
[error] both value jrowToInt in object Implicits of type =>
        (javadb.JRow, String) => Int
[error] and value jrowToDouble in object Implicits of type =>
        (javadb.JRow, String) => Double
[error] match expected type (javadb.JRow, String) => T
[error]  val fourValue1: Byte = row.get("four") // 本行不会被编译
```
我们遇到了两种可能错误中的一种。在这个例子中,由于作用域中存在一些隐式转换,同时, Byte 是如同 Int 或 Double 类型的数字,编译器因此并没有将其转换成两者中的任意一种类型,因为编译不允许出现二义性。但由于这两个函数抽取了太多的字节,无论如何都会抛出错误!

假设当前作用域中不存在任何隐式转换,你也会得到一个不同的错误。即使我们注释了对象 Implicit 中定义的三个隐式值,每次调用时也会出现下列错误:
```scala
[error] .../implicits/scala-database-api.scala:28:
  could not find implicit value for parameter toT: (javadb.JRow, String) => T
[error] val oneValue1: Int = row.get("one")
```

我们再回顾一下,**通过传入一个隐式参数以及只定义符合我们允许的类型所对应的隐式值,我们就可以对可用于参数化方法的类型进行了限定。**

顺便提一下,本示例受到了之前编写的 API 的启发。之前的 API 使得 JRow 能在更多时候与隐式函数关联上,这样我便可以嵌入“仿造”的或真实的 Cassandra 数据,其中的仿造数据用于测试。

#### 隐式证据
在上一节中,我们讨论了如何使用隐式对象对允许的类型进行限定,而这些允许的类型并不具有共同的超类。除此之外,我们还使用隐式对象执行 API 的相关工作。

有时候,我们只需要限定允许的类型,并不需要提供额外的处理。换句话说,我们需要“证据”证明提出的类型满足我们的需求。现在我们将讨论另外一种被称为隐式证据的相关技术来对允许的类型进行限定,而且这些类型无需继承某一共有的超类。

适用于所有可遍历(traversable)容器的 topMap 方法便是应用该技术的良好示例。我们回顾一下 Map 的构造函数,该函数接受键-值对(key-value pair)作为其输入参数,例如两元元组。假如我们拥有一连串的 pair,那么如果能通过一个操作基于这些 pair 值创建 Map对象岂不是更好?这便是 toMap 方法做的事情,不过我们还是有所顾虑,因为我们并不允许在调用 toMap 方法时输入一组非 pair 类型的序列。

TraversableOnce 是这样定义 toMap 方法的:
```scala
trait TraversableOnce[+A] ... {
  ...
  def toMap[T, U](implicit ev: <:<[A, (T, U)]): immutable.Map[T, U]
  ...
}
```
隐式参数 ev 便是我们的“证据”,它代表了我们必须实施的约束。 ev 运用了 Predef 中定义的名为 <:< 的类型,该名字取自于 <: 方法。 <: 方法同样也被用于限定类型参数,例如:A <: B 。

我们曾提及过,可以使用中缀表示法表示由两个类型参数所组成的类型,因此下列两种表达式是等价的:
```scala
<:<[A, B]
A <:< B
```
在 toMap 中, B 实际上是一个 pair:
```scala
<:<[A, (T, U)]
A <:< (T, U)
```

现在,假如我们希望将一个可遍历计划转换成 Map 对象,编译器会结合我们所需要的隐式证据 ev 值来进行判定,只有满足 A <: (T,U) 时才会执行操作。也就是说,编译器会检查A 是否是一个 pair,如果满足该条件,便会调用 toMap 方法并将可遍历的元素传递给 Map 构造器;假如 A 不是 pair 类型,编译器将会抛出错误:
```scala
scala> val l1 = List(1, 2, 3)
l1: List[Int] = List(1, 2, 3)

scala> l1.toMap
<console>:9: error: Cannot prove that Int <:< (T, U).
            l1.toMap
            ^

scala> val l2 = List("one" -> 1, "two" -> 2, "three" -> 3)
l2: List[(String, Int)] = List((one,1), (two,2), (three,3))

scala> l2.toMap
res3: scala.collection.immutable.Map[String,Int] =
Map(one -> 1, two -> 2, three -> 3)
```

因此,“证据”存在的意义仅仅是为了实施某一类型约束。我们无需定义一个隐式值来执行额外的自定义工作。

Predef 对象中还定义了一个名为 =:= 的“证据”类型,它可以证明两个类型之间的等价关系。但该类型并未得到广泛的应用。

#### 绕开类型擦除带来的限制
有了隐式“证据”之后,我们在计算时就可以不再使用隐式对象。更准确地说,我们只需要使用隐式“证据”证明输入满足某些特定的类型约束。

类型擦除(type erasure)会带来一些限制。但在接下来的示例中我们通过使用隐式对象提供的“证据”巧妙地绕开了这些限制。

由于历史原因, JVM“忘记”了为参数化类型提供类型参数。例如,我们在下面定义了重载方法,虽然这些方法具有相同的名称,但它们的类型签名却互不相同:
```scala
object C {
  def m(seq: Seq[Int]): Unit = println(s"Seq[Int]: $seq")
  def m(seq: Seq[String]): Unit = println(s"Seq[String]: $seq")
}
```
我们再来看看在 REPL 会话中运行这段代码会出现什么情况:
```scala
scala> :paste
// 进入粘贴模式(输入ctrl-D结束该模式)
object M {
  def m(seq: Seq[Int]): Unit = println(s"Seq[Int]: $seq")
  def m(seq: Seq[String]): Unit = println(s"Seq[String]: $seq")
}
<ctrl-d>
// 退出粘贴模式,现在进入解释执行模式。
<console>:8: error: double definition:
method m:(seq: Seq[String])Unit and
method m:(seq: Seq[Int])Unit at line 7
have same type after erasure: (seq: Seq)Unit
    def m(seq: Seq[String]): Unit = println(s"Seq[String]: $seq")
        ^
```
<ctrl-d> 说明我输入了 Ctrl-D,当然控制台并不会输出这一字符。

顺便提一下,假如在 object M 的作用域之外输入这两个方法的定义体,并且未使用 :paste模式,那么你将看不到任何报错信息。这是因为为了让客户使用方便,REPL 允许重定义类型、值和方法,而编译器在编译常规文件时则不允许这点。假如你忘记了 REPL 中有这样一个“便民”的设计,你也许会以为你成功定义了两个不同版本的 m 方法。而运行 :paste 模式时,编译器会把 Ctrl-D 出现前的所有输入视为一个待编译的普通文件。所以,正因为这些方法在字节码中是一样的,编译器不允许同时出现这些方法定义。不过,我们可以通过添加隐式参数来消除这些方法的二义性:
```scala
// src/main/scala/progscala2/implicits/implicit-erasure.sc
object M {
  implicit object IntMarker                                          // <1>
  implicit object StringMarker

  def m(seq: Seq[Int])(implicit i: IntMarker.type): Unit =           // <2>
    println(s"Seq[Int]: $seq")

  def m(seq: Seq[String])(implicit s: StringMarker.type): Unit =     // <3>
    println(s"Seq[String]: $seq")
}

import M._                                                           // <4>
m(List(1,2,3))
m(List("one", "two", "three"))
```
➊ 上面的代码中定义了两个具有特殊用途的隐式对象,这两个对象将用于解决由于类型擦除导致的方法二义性问题。  
➋ 重新定义输入参数为 Seq[Int] 类型的方法。现在该方法新增了第二个参数列表,新增 的 参 数 列 表 希 望 能 够 接 收 到 一 个 隐 式 IntMarker 对 象。 请 注 意 该 对 象 的 类 型 是IntMarker.type 。该类型引用了单例对象的类型!  
➌ 重新定义输入参数为 Seq[String] 的方法。  
➍ 导入并使用隐式值和方法。这些代码能够顺利通过编译并打印出正确的输出。

现在,即便发生了类型擦除,编译器还是会将这两个 m 方法视作不同的方法。你也许会思考为什么要发明新的类型,而不使用隐式 Int 和 String 值呢?我们并不推荐使用常用类型的隐式值。为什么呢?假设当前作用域中某一模块定义了 String 类型的一个隐式参数以及一个“默认”的隐式 String 类型值,之后该作用域的另外一个模块也定义了隐式 String参数,那么这两个隐式参数便会导致系统出错。首先,假设第二个模块并未定义“默认”隐式值,而希望用户自己能够定义适用于应用程序的隐式值。如果用户没有定义该隐式值,那么该模块便会使用其他模块的隐式值,这可能会导致无法预期的行为。如果用户定义了隐式值,那么这两个出现在相同作用域中的隐式值将会导致二义性,而编译器就会抛出错误。

第一种场景会导致无法预期的行为发生,而第二类场景则会立即触发错误。比较安全的做法是减少使用隐式参数及隐式值,改用那些专为此目的设计的特有类型。

**尽量避免对 Int 和 String 那样的常用类型使用隐式参数及其对应值。因为与其他类型相比,这些类型更有可能会在多处定义其对应的隐式对象。假如这些隐式定义被导入到相同的作用域内,便会导致程序 bug 或编译错误。**

我们会在第 14 章中更详细地讨论类型擦除的相关内容。

#### 改善报错信息
我们暂时先回到集合 API 和 CanBuildFrom 构造器的相关示例中。如果试图对某一个未定义对应 CanBuildFrom 的目标类型调用 map 方法,那会发生什么呢?
```scala
scala> case class ListWrapper(list: List[Int])
defined class ListWrapper

scala> List(1,2,3).map[Int,ListWrapper](_ * 2)
<console>:10: error: Cannot construct a collection of type ListWrapper
with elements of type Int based on a collection of type List[Int].
        List(1,2,3).map[Int,ListWrapper](_*2)
                                        ^
```
上述代码在 map 方法中指定了明确的类型注解 map[Int,ListWrapper] ,这确保了输出的对象类型是我们所希望的 ListWrapper 类型,而不是默认的 List[Int] 类型。而该注解同时也触发了我们预期引发的错误。请注意描述性的错误信息:Cannot construct a collectionof type ListWrapper with elements of type Int based on a collection of type List[Int](如果输入类型为 List[Int] ,我们无法通过类型是 Int 类型的元素构造类型为 ListWrapper 的集合)。这并不是通常情况下编译器因为无法找到某一隐式参数的隐式值时所产生的默认信息。实际上,在声明 CanBuildFrom 时指定了一个名为 scala.annotation.implicitNotFound的 注 解(annotation, 与 Java 的 annotation 相 似)。该注解指定了那些错误信息的格式字符串(如果想了解更多 Scala 注解相关的内容,请查阅 23.2 节)。 CanBuildFrom 的声明体如下所示:
```scala
@implicitNotFound(msg =
  "Cannot construct a collection of type ${To} with elements of type ${Elem}" +
  " based on a collection of type ${From}.")
trait CanBuildFrom[-From, -Elem, +To] {...}
                                        ^
```

你只能对那些专为满足隐式参数而定义的、用作隐式值的类型使用这类注解。而这些注解无法用于像 SRow.get[T] 那样的接受隐式参数输入的方法之上。

这也解释了为什么构建隐式时应该使用自定义类型,而不应使用更为常见的类型,比如说Int 类型、 String 类型甚至是像 SRow 示例中出现的 (JRow, String) => T 函数类型。使用自定义类型,你能为用户提供更有用的错误信息。

#### 虚类型
我们之前已经学习了像 CanBuildFrom 那种可以添加行为的隐式参数。也使用了已经定义好的可作为 API 调用时的隐式值, 如:作用于集合的 toMap 方法和 <:< 类型的隐式实例。

**接下来我们要进行的步骤是移除所有的实例,仅仅留下需要的类型。这类定义好的没有任何实例的类型被称为虚类型(phantom type)。**尽管虚类型的名字听起来较为花哨,不过它仅表明我们只关心类型本身。虚函数便是作为这样一个标志而存在的,表明我们不会使用该类型的任何实例。

尽管我们即将谈论的虚类型与隐式没有任何关系,不过它却能用在我们之前解决的设计问题上。

对于定义必须按照某一特定顺序执行的工作流而言,虚类型作用很大。我们举一个简化版的工资计算器的例子。根据美国税法,在计算工资税之前,保险基金以及某些退休存款(401k)账户可以作为抵税项先从工资中扣除。因此,工资计算器必须首先执行“扣税前”的减项操作,然后进行扣税,最后计算器会扣除扣税后的其他减项并算出净收入。

下面便是该示例可能的一种实现:
```scala
// src/main/scala/progscala2/implicits/phantom-types.scala

// A workflow for payroll calculations.
// 工资计算的工作流程。

package progscala.implicits.payroll

sealed trait PreTaxDeductions
sealed trait PostTaxDeductions
sealed trait Final

// For simplicity, use Float for money. Not recommended...
// 为了简单起见,此处使用Float类型表示金额。不推荐大家这样做......
case class Employee(
  name: String,
  annualSalary: Float,
  taxRate: Float,  // For simplicity, just 1 rate covering all taxes.// 为了简化起见, 所有的税种税率相同。
  insurancePremiumsPerPayPeriod: Float,
  _401kDeductionRate: Float,  // A pretax, retirement savings plan in the USA.// 税前扣除项,美国退休储蓄计划扣款。
  postTaxDeductions: Float)

case class Pay[Step](employee: Employee, netPay: Float)

object Payroll {
  // Biweekly paychecks. Assume exactly 52 weeks/year for simplicity.
  // 每两周发一次薪水。为了简化问题,我们认定每年正好有52周。
  def start(employee: Employee): Pay[PreTaxDeductions] =
    Pay[PreTaxDeductions](employee, employee.annualSalary / 26.0F)

  def minusInsurance(pay: Pay[PreTaxDeductions]): Pay[PreTaxDeductions] = {
    val newNet = pay.netPay - pay.employee.insurancePremiumsPerPayPeriod
    pay copy (netPay = newNet)
  }

  def minus401k(pay: Pay[PreTaxDeductions]): Pay[PreTaxDeductions] = {
    val newNet = pay.netPay - (pay.employee._401kDeductionRate * pay.netPay)
    pay copy (netPay = newNet)
  }

  def minusTax(pay: Pay[PreTaxDeductions]): Pay[PostTaxDeductions] = {
    val newNet = pay.netPay - (pay.employee.taxRate * pay.netPay)
    pay copy (netPay = newNet)
  }

  def minusFinalDeductions(pay: Pay[PostTaxDeductions]): Pay[Final] = {
    val newNet = pay.netPay - pay.employee.postTaxDeductions
    pay copy (netPay = newNet)
  }
}

object CalculatePayroll {
  def main(args: Array[String]): Unit = {
    val e = Employee("Buck Trends", 100000.0F, 0.25F, 200F, 0.10F, 0.05F)
    val pay1 = Payroll start e
    // 401K and insurance can be calculated in either order.
    // 401K和保险扣除的顺序可以交换
    val pay2 = Payroll minus401k pay1
    val pay3 = Payroll minusInsurance pay2
    val pay4 = Payroll minusTax pay3
    val pay  = Payroll minusFinalDeductions pay4
    val twoWeekGross = e.annualSalary / 26.0F
    val twoWeekNet   = pay.netPay
    val percent      = (twoWeekNet / twoWeekGross) * 100
    println(s"For ${e.name}, the gross vs. net pay every 2 weeks is:")
    println(
      f"  $$${twoWeekGross}%.2f vs. $$${twoWeekNet}%.2f or ${percent}%.1f%%")
  }
}
```
代码已经通过了 sbt 的编译,因此可以在 sbt 命令行下执行程序:
```scala
> run-main progscala.implicits.payroll.CalculatePayroll
[info] Running progscala.implicits.payroll.CalculatePayroll
For Buck Trends, the gross vs. net pay every 2 weeks is:
  $3846.15 vs. $2446.10 or 63.6%
```
**请注意,这些密封特征(sealed trait)本身不包含任何数据,而且没有实现这些 trait 的类。由于这些 trait 被“密封”了,我们无法在其他文件中实现这些 trait,因此,这些 trait 只能起到标志的作用。**

Pay 类型将这些标志用作类型参数,而这些参数也被当作标记,调用 Payroll 类的每一个方法均需要传入 Pay[Step] 对象,该对象包含的 Step 参数表示了当前执行的步骤。这也是我们在调用 minus401k 方法时无法传入 Pay[PostTaxDeductions] 对象的原因。因此,这些API 能够确保税务规定的执行。

CalculatePayroll 对象演示了这些 API 的使用方法,假如你尝试修改 API 的调用顺序,如在调用 Payroll.minusTax 方法之前调用 Payroll.minusFinalDeductions 方法,那么编译将会抛出错误。

请注意,本示例结尾处调用的 println 方法使用了可插入字符串,该字符串与我们之前在3.13 节讨论的示例非常相近。

事实上,上述代码中的 main 函数不是十分简洁,这种复杂性也破坏了代码的美感。为了解决这个问题,我们引入了微软的函数式编程语言——F# 语言中的“管道”操作符。下面的示例代码改编自 James Iry 的一篇博文(http://james-iry.blogspot.ch/2010/10/phantom-types-in-haskell-and-scala.html)。
```scala
// src/main/scala/progscala2/implicits/phantom-types-pipeline.scala
// A nicer way of driving the payroll workflow. Adapted from
// http://james-iry.blogspot.ch/2010/10/phantom-types-in-haskell-and-scala.html
// which was inspired by F# and Haskell.
package progscala.implicits.payroll

object Pipeline {
  implicit class toPiped[V](value:V) {
    def |>[R] (f : V => R) = f(value)
  }
}

object CalculatePayroll2 {
  def main(args: Array[String]): Unit = {
    import Pipeline._
    import Payroll._

    val e = Employee("Buck Trends", 100000.0F, 0.25F, 200F, 0.10F, 0.05F)
    val pay = start(e) |>
      minus401k        |>
      minusInsurance   |>
      minusTax         |>
      minusFinalDeductions
    val twoWeekGross = e.annualSalary / 26.0F
    val twoWeekNet   = pay.netPay
    val percent      = (twoWeekNet / twoWeekGross) * 100
    println(s"For ${e.name}, the gross vs. net pay every 2 weeks is:")
    println(
      f"  $$${twoWeekGross}%.2f vs. $$${twoWeekNet}%.2f or ${percent}%.1f%%")
  }
}
```

main 方法中包含了更为简洁的一组操作步骤,这些操作均调用了 Payroll 方法。值得注意的是,尽管管道操作符 |> 看上去很酷,但它其实只是重排了表达式中各个标记的次序。例如: |> 操作符对下列表达式就进行了转化:
```scala
pay1 |> Payroll.minus401k
```
转化之后形成了下列表达式:
```scala
Payroll.minus401k(pay1)
```

#### 隐式参数遵循的规则
回顾下隐式参数,下面的编注栏中列出了隐式参数应遵循的通用规则。

* (1) 只有最后一个参数列表中允许出现隐式参数,这也适用于只有一个参数列表的情况。  
* (2) implicit 关键字必须出现在参数列表的最左边,而且只能出现一次。列表中出现在implicit 关键字之后的参数都不是“非隐式”的。  
* (3) 假如参数列表以 implicit 关键字开头,那么所有的参数都是隐式的。  

下面示例违背了上述规则:
```scala
//错误的
class Bad1 {
  def m(i: Int, implicit s: String) = "boo"
}

//错误的
class Bad2 {
  def m(i: Int)(implicit s: String)(implicit d: Double) = "boo"
}

//正确的
class Good1 {
  def m(i: Int)(implicit s: String, d: Double) = "boo"
}

//正确的
class Good2 {
  def m(implicit i: Int, s: String, d: Double) = "boo"
}
```

### 隐式转换
在 2.8.8 节,我们学习了创建 pair(二元组) 的一些方法:
```scala
(1, "one")
1 -> "one"
1 → "one"          // 用 → 替代 ->
Tuple2(1, "one")
Pair(1, "one")     // Scala 2.11不推荐使用这种写法
```
本文不会对 (a, b) 和 a -> b 这两类字面量格式的 Scala 语法进行讲解。

在初始化 Map 对象时,我们经常使用 a -> b (或等价的 a → b )这种写法:
```scala
scala> Map("one" -> 1, "two" -> 2)
res0: scala.collection.immutable.Map[String,Int] = Map(one -> 1, two -> 2)
```

上述代码中调用了 Map.apply 方法,而该方法的输入是一组可变数量的 pair 对象:
```scala
def apply[A, B](elems: (A, B)*): Map[A, B]
```
事实上,Scala 根本不知道 a -> b 意味着什么,因此上述方法并非没有意义。这种“字面量”格式实际上运用了方法 -> 和一个特殊的 Scala 特性——隐式转换。通过运用隐式转换,我们可以在任意两种类型值之间插入函数 -> 。与此同时,由于 a -> b 并不是元组的字面量语法,因此 Scala 必须通过某些方式将该表达式转化为元组 (a, b) 。

很明显,我们首先需要定义方法 -> ,但是应该在哪儿定义该方法呢? 我们希望该方法能够适用于各种可能的元组首元素的类型。即便我们能够对所有需要添加该方法的类型进行编辑,我们也不应该这样做,编辑所有的类型既不实际也不明智。

定义该方法的技巧是使用一个具有 -> 方法的“封装”对象,Scala 已经在 Predef 对象中定义了该对象:
```scala
implicit final class ArrowAssoc[A](val self: A) {
  def -> [B](y: B): Tuple2[A, B] = Tuple2(self, y)
}
```
(为了更清楚地说明问题,我在此处省略了实际声明体中的一些不重要的细节 )。与 Java 类似,此处的 final 关键字表明了我们无法创建 ArrowAssoc 的子类声明。

我们可以依照下列代码构造 Map 对象:
```scala
scala> Map(new ArrowAssoc("one") -> 1, new ArrowAssoc("two") -> 2)
res0: scala.collection.immutable.Map[String,Int] = Map(one -> 1, two -> 2)
```
这种写法在简洁程度上不如直接使用 Map( ("one", 1), ("two", 2) ) 。不过,这种写法能实现我们想要的部分功能。 ArrowAssoc 类能够接受任何类型的对象,当执行 -> 方法时,将返回一个 pair 对象。

在这个示例中, implicit 关键字又一次起作用了。由于 ArrowAssoc 类被声明为 implicit类,因此编译器将执行下列逻辑。

* (1) 编译器发现我们试图对 String 对象执行 -> 方法(例如 “one” -> 1 )。  
* (2) 由于 String 未定义 -> 方法,编译器将检查当前作用域中是否存在定义了该方法的隐式转换。  
* (3) 编译器发现了 ArrowAssoc 类。  
* (4) 编译器将创建 ArrowAssoc 对象,并向其传入 one 字符串。  
* (5) 之后,编译器将解析表达式中的 -> 1 部分代码,并确认了整个表达式的类型与 Map.apply 方法的预期类型相吻合,即两者均为 pair 实例。  

如果希望执行隐式转换,那么在声明时必须使用 implicit 关键字,能够执行隐式转换的无外乎两类:构造方法中只接受单一参数的类型或者是只接受单一参数的方法。

在 Scala 2.10 诞生之前,如果想要实现隐式转换,就必须定义一个不含 implicit 关键字的封装类,并且要在封装类中定义一个将执行转换的隐式方法。由于这种两段式的定义方式毫无意义,因此现在我们可以直接将该类标注为隐式类并清除该方法。例如, ArrowAssoc看上去就好像 Scala 2.10 版本之前的封装类( any2ArrowAssoc 方法仍然存在,不过现在已经不建议使用了)。
```scala
final class ArrowAssoc[A](val self: A) {
  def -> [B](y: B): Tuple2[A, B] = Tuple2(self, y)
}
...
implicit def any2ArrowAssoc[A](x: A): ArrowAssoc[A] = new ArrowAssoc(x)
```

**尽管隐式方法仍然被使用,但是我们现在只有在将某类型转换为另一个已经存在的类型时才会使用这些方法,而且这些隐式方法并未使用 implicit 关键字进行声明。**

我们之前在讲解虚类型(phantom type)时曾定义了管道操作,这也是隐式转换的又一示例:
```scala
... pay1 |> Payroll.minus401k ...
```
滥用隐式方法会导致难以调试的诡异行为。因此从 Scala 2.10 开始,隐式方法变成了Scala 的可选特性。假如你希望使用这一特性,你应该通过 import 语句 import scala.langage.implicitConversions 开启这一特性,你也可以使用全局的编译器选项 -language:implictConversions 开启该特性。

以下是编译器进行查找和使用转换方法时的查询规则。

* (1) 假如调用的对象和方法成功通过了组合类型检查,那么类型转换不会被执行。  
* (2) 编译器只会考虑使用了 implicit 关键字的类和方法。  
* (3) 编译器只会考虑当前作用域内的隐式类,隐式方法,以及目标类型的伴生对象中定义的隐式方法(本文后续部分将讲讨论这种情况)。  
* (4) 隐式方法无法串行处理,我们无法通过一个中间类型,使用串行的隐式方法将起始类型转换成目标类型。编译器执行隐式转换时只会考虑那些接受单一类型实例输入且返回目标类型实例的方法。  
* (5) 假如当前适用多条转换方法,那么将不会执行转换操作。编译器要求有且必须只有一条满足条件的隐式方法,以免产生二义性。  

规则 3 中提到了伴生对象中定义的隐式方法,该规则具有以下含义。首先,假如隐式转换出现在当前作用域之外,那么便不会执行该隐式转换。假如在当前作用域内,这意味着隐式转换是在封闭作用域内声明的,或者当前作用域已经导入了隐式转换所在的其他作用域,比方说导入了其他作用域中某一个定义了一些隐式转换的对象。

**不过当需要执行转换时,假如转换的目标类型的伴生对象中定义了转换方法,那么编译器会自动导入伴生对象作用域,并最后查找该作用域。**参见下面的示例:
```scala
// src/main/scala/progscala2/implicits/implicit-conversions-resolution.sc
import scala.language.implicitConversions

// WARNING: You must :paste mode in the REPL for the following.
// Using :load won't compile the two definitions together!
case class Foo(s: String)
object Foo {
  implicit def fromString(s: String): Foo = Foo(s)
}

class O {
  def m1(foo: Foo) = println(foo)
  def m(s: String) = m1(s)
}
```
Foo 类型的伴生对象定义了基于 String 类型的转换。而 0.m 方法试图在调用 0.m1 方法时传入 String 类型,但 0.m1 方法却期望输入 Foo 对象。尽管我们并未明确地将 Foo 伴生对象导入当前的作用域,编译器还是能够发现 Foo.fromString 转换方法的存在。

不 过, 假 如 当 前 作 用 域 中 存 在 另 外 一 个 Foo 转 换 方 法, 该 转 换 方 法 将 会 覆 盖 Foo.fromString 转换:
```scala
// src/main/scala/progscala2/implicits/implicit-conversions-resolution2.sc
import scala.language.implicitConversions

case class Foo(s: String)
object Foo {
  implicit def fromString(s: String): Foo = Foo(s)
}

implicit def overridingConversion(s: String): Foo = Foo("Boo: "+s)

class O {
  def m1(foo: Foo) = println(foo)
  def m(s: String) = m1(s)
}
```
现在,编译器将使用覆盖后的转换方法。

**我们之前提到过,除了那些在伴生对象中定义的隐式值和转换方法之外,我们推荐将隐式值和转换方法放到一个名为 implicits 的特殊包或名为 Implicits 的对象中。这样做的好处是:读者能更清楚地知道是哪个 import 语句导入了代码使用的自定义隐式。**

最 后, 请 注 意 Scala 为 像 String 和 Array 这 样 的 Java 类 型 提 供 了 一 些 隐 式 封 装 类型。例如:下面代码中出现的方法看上去像是 String 类方法,但是这些方法实际上是由 WrappedString 类 型实现的:
```scala
scala> val s = "Programming Scala"
s: String = Programming Scala

scala> s.reverse
res0: String = alacS gnimmargorP

scala> s.capitalize
res1: String = Programming Scala

scala> s.foreach(c => print(s"$c-"))
P-r-o-g-r-a-m-m-i-n-g- -S-c-a-l-a-
```

在 Scala 自带的“封装”类型中定义的隐式转换会一直出现在当前作用域中。这些隐式转换更确切的讲被定义在 Predef 中。

我们之前用过的 Range 类型也是一类“封装”类型。例如:代码 1 to 100 by 3 代表从 1 到 100每隔 3 个数取一个整数。你现在应该能猜测出 to 、 by 这样的字样以及 scala 独有的 util 单词都是封装对象中的方法,它们并不是 Scala 关键字。例如, scala.runtime.RichInt (http://www.scala-lang.org/api/current/scala/runtime/RichInt.html)封装了 Int 类型,它包含这些方法。Scala 在同一个包中定义了适用于其他数值类型的类似的封装类型: RichLong 、 RichFloat 、 RichDouble和 RichChar 。而 scala.math.BigInt 和scala.math.BigDecimal 它们本身便是 Java 同名类的封装类型,因此它们没有自己的封装类型。它们自己本身便可以实现 to 、until 和 by 方法。

#### 构建独有的字符串插入器
我们再来学习最后一个隐式转换的示例,该示例允许我们通过隐式转换定义我们自己独有的字符串插入器。我们回顾一下之前在 3.13 节学到的内容:Scala 提供了一些内置的方法通过插入方式对字符串进行格式化,例如:
```scala
val name = ("Buck", "Trends")
println(s"Hello, ${name._1} ${name._2}")
```
当编译器看到像 x"foo bar" 这样的表达式时,它会查找 scala.StringContext 中定义的 x 方法。上述代码的最后一行可以被转换成:
```scala
StringContext("Hello, ", " ", "").s(name._1, name._2)
```
传递给 StringContext.apply 方法的参数其实是 ${...} 表达式中抽取出的各个部分。传递给 s 的参数则是抽取后的表达式。(请提供变量值,对上述代码进行试验) StringContext中还定义了 f 和 raw 方法,这些方法能够帮助 f 格式以及 raw 格式的插入器工作。

通过使用隐式转换我们可以为 StringContext 添加新的方法,对其进行“扩展”,进而定义属于自己的插入器。我们对 StringContext Scaladoc 页面(http://www.scala-lang.org/api/current/scala/StringContext.html)中出现的示例进行扩充,编写一个能够将简单的 JSON字 符 串 转 化 为 scala.util.parsing.json.JSONObject 对 象的插入器。

为了简化,我们会做一些假设。首先,我们不会对数组或嵌套对象进行处理,只处理像{"a": "A", "b": 123, "c": 3.14159} 这样的扁平的 JSON 表达式。其次,我们要求 JSON的 key 都是固定值,而 value 值则是指定的可插入参数,例如: {"a": $a, "b": $b, "c":$c} 。假如我们需要使用该插入器生成 value 值不同但结构相似的 JSON 对象,第二条限制便是合理的。插入器实现代码如下:
```scala
// src/main/scala/progscala2/implicits/custom-string-interpolator.sc
import scala.util.parsing.json._

object Interpolators {
  implicit class jsonForStringContext(val sc: StringContext) {       // <1>
    def json(values: Any*): JSONObject = {                           // <2>
      val keyRE = """^[\s{,]*(\S+):\s*""".r                          // <3>
      val keys = sc.parts map {                                      // <4>
        case keyRE(key) => key
        case str => str
      }
      val kvs = keys zip values                                      // <5>
      JSONObject(kvs.toMap)                                          // <6>
    }
  }
}

import Interpolators._

val name = "Dean Wampler"
val book = "Programming Scala, Second Edition"

val jsonobj = json"{name: $name, book: $book}"                       // <7>
println(jsonobj)
```
➊ 为了限定隐式类的作用域,必须在对象内定义隐式类(出于安全的考虑)。类定义之后出现的 import 语句将该实现类导入到代码所在的作用域中。  
➋ json 方法。该方法的输入是字符串中嵌套的参数,该方法返回构造好的 scala.util.parsing.json.JSONObject 对 象。  
➌ 用于从字符串片段中抽取 key 名称的正则表达式。  
➍ 使用正则表达式从字符片段各个部分中抽取出 key 的名称。假如正则表达式不匹配,那么将使用整个字符串作为 key 值。不过在这种情况下抛出错误可能是一个更好的选择,因为这样便能避免出现无效的 key 字符串。  
➎ 将 key 值和 value 值一并“压缩”成健-值对集合。我们在解释完代码之后会对 zip 方法进行讨论。  
➏ 使用健值对构建 Map 对象,并使用 Map 对象构造 JsonObject 对象。  
➐ 使用我们自己的字符串插入器,使用起来与内置插入器无异。   

自 定 义 的 字 符 串 插 入 器 无 需 像 s 、 f 和 raw 插 入 器 那 样 返 回 String 类 型。 我 们 会 返 回JSONObject 类型。因此,自定义字符串插入器可以用作由字符串中封装的数据所驱动的实例工厂。

就像拉链一样,集合中的 zip 方法能很方便地将两个集合中的值缝合在一起。如下所示:
```scala
scala> val keys = List("a", "b", "c", "d")
keys: List[String] = List(a, b, c, d)

scala> val values = List("A", 123, 3.14159)
values: List[Any] = List(A, 123, 3.14159)

scala> val keysValues = keys zip values
keysValues: List[(String, Any)] = List((a,A), (b,123), (c,3.14159))
```

合并后的集合中的元素类型为二元元祖(如 key1 、 value1 等)。需要注意的是,由于某一列表比另一个列表长,因此列表尾部多余的元素便会被丢弃掉。JSON 字符串中字符串片段数量比值参数的数量多一个,但我们实际上希望出现这类行为,因为我们并不需要字符串尾部的最后一个片段。

下面列出了在 REPL 会话中运行示例代码后,最后两行中的输出:
```scala
scala> val jsonobj = json"{name: $name, book: $book}"
jsonobj: scala.util.parsing.json.JSONObject = \
{"name" : "Dean Wampler", "book" : "Programming Scala, Second Edition"}

scala> println(jsonobj)
{"name" : "Dean Wampler", "book" : "Programming Scala, Second Edition"}
```

#### 表达式问题
我们再思考一下之前已经解决过的问题:我们已经能够在不修改任何相关源代码的情况下成功地为所有类型添加新的方法。

在不修改源代码的情况下扩展模块的期望被称为表达式问题(expression problem)。这一概念 是 由 Philip Wadler 创造的。

面向对象编程通过子类型化(subtyping)解决了这一问题,更精确地讲是子类型多态(subtype polymorphism)。当需要对某些行为进行修改时,我们会首先编写抽象体,之后使用抽象体的继承类修改行为。Bertrand Meyer 提出的开 / 闭准则(open/closed principle)对OOP 这一方法进行了描述,在这一准则中,基类声明了抽象的行为,而子类型则在不修改基类型的情况下对抽象行为进行适当的修改。

Scala 当然也支持这一技术,不过这项技术是有弊端的。我们是否应该在类继承关系中的父类型中定义这些行为呢?假如只有少数几种情况下我们才需要这种行为,而大多数情况下该行为只是客户端代码加载时的负担?

这种方法有可能会成为负担。首先,这些额外的无用代码会占据系统资源。尽管有些应用并不会为此影响,但有些成熟代码会不可避免地出现很多损耗。其次,我们为此需要对大多数定义的行为进行一次次的修改。一旦对这些无用行为进行修改,即便不需要使用这一行为的客户端代码也需要进行多余的修改。

这一问题引发了单一职责原则(single responsibility principle)这一经典设计原则。该原则鼓励我们在定义抽象以及实现体时,只提供某一单一的行为。

在实际场合中,有时我们需要某一对象来把其他行为组合起来。例如,服务对象往往需要“混入”日志信息这一功能。正如我们在 3.14 节中展示的那样,Scala 很容易实现这些混入trait。我们甚至可以在声明对象时为其指定 trait。

动态类型语言通常都提供元编程(metaprogramming)功能,该功能允许用户在元编程运行的环境下,不必修改源代码就可以修改类。对于类导致的问题,尤其是这些类定义了几乎不被使用的行为,这一方法具有一定的效果。不幸的是,对于大多数动态语言而言,运行时对类型做的任一修改都会作用于全局,因此全局内的所有用户都会被影响。

通过使用 Scala 的隐式转换特征,我们可以得到另外一种通过静态类型实现元编程的方法,我们称之为类型类(type class)。Haskell 率先提出这一概念,可以参考文章“A Gentle Introduction to Haskell”。类型类这一名称源于Haskell,请不要将其与 Scala 常用类所混淆(如果某对象属于某类型类,那么它必须实现类型类所定义的行为,与通常意义上的类相比,类型类更接近于接口)。

### 类型类模式
**由于我们可以即兴地为类型添加行为,因此使用类型类可以帮助我们避免创建像 Java 中 Object 那样的抽象体**,例如“厨房水槽”这样的抽象体。**Scala 的 pair 构造语法 -> 便很好地说明了这点。**回想一下,我们并没有修改这些类型;我们只是通过隐式机制将对象封装到某个提供了我们所需行为的类型中。修改之后的结果与我们直接修改类型源代码的结果一样。

Scala 作为一门 JVM 语言,也继承了 Java 中无所不在的 Object.toString 方法。Java 默认的 toString 方法只会显示类型名称和对象在 JVM 堆中的地址,因此它本身并没有多大的价值。而 Scala 在 case 类中使用的语法则不同,该语法更加有用,也更具可读性。有时候打印出像 JSON 或 XML 样的具有机器可读性的格式是很有价值的。通过使用隐式转换,我们可以为任何类型添加 toJSON 和 toXML 方法。如果对象中未定义 toString 方法,我们也能通过隐式转换定义该方法。

Haskell 语言中的类型类能定义等价于接口的类型,之后我们便能实现各种具体类型。**Scala 中的类型类模式(type class pattern)新增了接口部分,这一功能是之前的隐式转换示例所未提供的。**

让我们阅读一下 toJSON 类型类的某一实现:
```scala
// src/main/scala/progscala2/implicits/toJSON-type-class.sc

case class Address(street: String, city: String)
case class Person(name: String, address: Address)

trait ToJSON {
  def toJSON(level: Int = 0): String

  val INDENTATION = "  "
  def indentation(level: Int = 0): (String,String) = 
    (INDENTATION * level, INDENTATION * (level+1))
}

implicit class AddressToJSON(address: Address) extends ToJSON {
  def toJSON(level: Int = 0): String = {
    val (outdent, indent) = indentation(level)
    s"""{
      |${indent}"street": "${address.street}", 
      |${indent}"city":   "${address.city}"
      |$outdent}""".stripMargin
  }
}

implicit class PersonToJSON(person: Person) extends ToJSON {
  def toJSON(level: Int = 0): String = {
    val (outdent, indent) = indentation(level)
    s"""{
      |${indent}"name":    "${person.name}", 
      |${indent}"address": ${person.address.toJSON(level + 1)} 
      |$outdent}""".stripMargin
  }
}

val a = Address("1 Scala Lane", "Anytown")
val p = Person("Buck Trends", a)

println(a.toJSON())
println()
println(p.toJSON())
```
为了简化起见, Person 和 Address 类型中只包含了少量的字段,同时我们希望将对象格式化为多行表示的 JSON 字符串,而不是 scala.util.parsing.json 包里定义的对象类型。

我们在 ToJSON trait 中定义了默认的缩进字符串,也定义了用于计算字段实际缩进长度的方法以及 JSON 对象中包含的闭合括号 {...} 。 toJSON 方法的输入参数指定了当前的缩进级别;也就是说,缩进多少个 INDENTATION 单元。由于 toJSON 方法要求输入这一参数,客户程序就必须提供空括号或者其他缩进级别。需要注意的是输入的缩进级别是用双引号包裹的字符串值,而不是整数值。

上述脚本输出如下:
```js
{
  "street": "1 Scala Lane", 
  "city":   "Anytown"
}

{
  "name":    "Buck Trends", 
  "address": {
    "street": "1 Scala Lane", 
    "city":   "Anytown"
  } 
}
```

**Scala 不允许同时使用 implict 和 case 关键字。**也就是说,隐式类不能同时是一个 case 类。由于 case 类不会执行通过隐式所自动生成的额外代码,因此隐式 case 类本身也就没有任何意义。可见隐式类的用途非常窄。

与面向对象继承关系中常出现的子类型多态(subtype polymorphism)不同,toJSON 方法所提供的多态行为并未绑定类型系统。因此类型类提供的这一功能也被称为特设多态(ad hoc polymorphism)。我们之前在 2.13 节中介绍过第三类多态:参数化多态(paremetric polymorphism)。在这一类多态中,像 Seq[A] 这样的容器的行为因 A 类型不同而不同。

我们常会有这样的错觉:如果定义了一组类,每个类只提供某一特定的行为,对于大多数客户而言,这些特定的行为并没有什么作用。而类型类模式最适合用于这种情况,对这组类应用类型类模式能够使客户从中获益。善用该模式能够在维护单一职责原则的同时平衡多个客户的需求。

### 隐式所导致的技术问题
那么隐式有哪些问题呢?在定义类型时,为什么不将类型定义为仅比字段包(字段包有时候又称为贫血类型,anemic type)稍丰富一点,只提供非常少的行为的类型呢?然后再使用类型类添加所有的行为呢?

首先,你需要花时间编写用于定义隐式的额外代码,而且编译器也必须费力处理隐式。因此,编译那些大量应用隐式的项目需要耗费很多的时间。

隐式转换同样会造成额外的运行开销,这是因为封装类型会引入额外的中间层。尽管编译器会内联某些方法调用,而且 JVM 也会执行一些额外的优化,但是这样还是会有些额外开销。我们会在第 14 章谈论值类型,用于在编译期间消除值类型的额外运行开销。

当隐式特征与其他 Scala 特征,尤其是子类型特征发生交集时,会产生一些技术问题(关于子类型特征的完整内容,请阅读 scala-debate email 组的讨论贴)。

我们用一个简单示例来说明这一点:
```scala
// src/main/scala/progscala2/implicits/type-classes-subtyping.sc

trait Stringizer[+T] {
  def stringize: String
}

implicit class AnyStringizer(a: Any) extends Stringizer[Any] {
  def stringize: String = a match {
    case s: String => s
    case i: Int => (i*10).toString
    case f: Float => (f*10.1).toString
    case other => 
      throw new UnsupportedOperationException(s"Can't stringize $other")
  }
}

val list: List[Any] = List(1, 2.2F, "three", 'symbol)

list foreach { (x:Any) => 
  try {
    println(s"$x: ${x.stringize}")
  } catch {
    case e: java.lang.UnsupportedOperationException => println(e)
  }
}
```
**我们定义了一个名为 Stringizer 的抽象体。如果按照之前 ToJSON 示例的做法,我们会为所有我们希望能字符串化的类型创建隐式类。这本身就是一个问题。如果我们希望处理一组不同的类型实例,我们只能在 list 类型的 map 方法内隐式地传入一个 Stringizer 实例。因此,我们就必须定义一个 AnyStringerize 类,该类知道如何对我们已知的所有类型进行处理。这些类型甚至还包含用于抛出异常的 default 子句。**

**这种实现方式非常不美观,同时也违背了面向对象编程中的一条核心规则——你不应该使用 switch 语句对可能发生变化的类型进行判断。相反,你应该利用多态分发任务,这类似于 toString 方法在 Scala 和 Java 语言中的运作方式。**

如果你想更深入地了解 ToJSON 方法作用于一组对象的具体方法,请查看相关示例代码:implicits/type-classes-subtyping2.sc 。

最后,我将列出帮助我们避免某些潜在问题的一些技巧。

无论何时都要为隐式转换方法指定返回类型。否则,类型推导推断出的返回类型可能会导致预料之外的结果。

另外,虽然编译器会执行一些“方便”用户的转换。但是目前来看这些转换带来的麻烦多过益处(以后推出的 Scala 也许会修改这些行为)。

首先,假如你为某一类型定义方法 + ,并试图将该方法应用到某一不属于该类型的实例上,那么编译器会调用该实例的 toString 方法,这样一来便能执行 String 类型的 + 操作(合并字符串操作)。这可以解释某些特定情况下出现像 String 是错误类型的奇怪错误。

与此同时,如果有必要的话,编译器会将方法的输入参数自动组合成一个元组。有时候这一行为会给人造成困扰。幸运的是,Scala 2.11 现在会抛出警告信息。
```scala
scala> def m(pair:Tuple2[Int,String]) = println(pair)

scala> m(1, "two")
<console>:9: warning: Adapting argument list by creating a 2-tuple:
this may not be what you want.
    signature: m(pair: (Int, String)): Unit
given arguments: 1, "two"
after adaptation: m((1, "two"): (Int, String))
        m(1,"two")
         ^
(1,two)
```
也许这个时候你已经有点晕乎了,不过这也没关系。我希望你能清楚认识到隐式是 Scala中的一门强大的工具,而且在使用隐式时也能保持头脑清醒。

### 隐式解析规则
Scala 查找隐式时,会遵照一组复杂的搜寻规则,其中某些规则的设计初衷是为了解决潜在的二义性。

尽管根据隐式所处的情景不同,会存在隐式方法、隐式值和隐式类这三类隐式。在下面的讨论中,我将使用“值”一词来表示隐式。

* Scala 会解析无须输入前缀路径的类型兼容隐式值。换句话说,隐式值定义在相同作用域中。例如:隐式值定义在相同代码块中,隐式值定义在相同类型中,隐式值定义在伴生对象中(如果存在的话),或者定义在父类型中。  
* Scala 会解析那些导入到当前作用域的隐式值(这也无须输入前缀路径)。  

第二条规则中提到的导入隐式值,其优先级高于已经在当前作用域的隐式值。

**有的时候,会存在多个可能的类型兼容的匹配,这时 Scala 将挑选匹配度最高的隐式。举个例子,如果隐式参数类型是 Foo 类型,而当前作用域中既存在 Foo 类型的隐式值又存在AnyRef 类型的隐式值,那么 Scala 会挑选类型为 Foo 的隐式值。**

如果两个或多个隐式值可能引发歧义(例如:它们具有相同的类型),编译错误会被触发。

Scala 库中定义的隐式常常会被编译器加载到作用域中,而其他语言库中定义的隐式则需要通过 import 语句来加载进作用域。接下来我们将讨论这些被编译器加载的隐式。

### Scala 内置的各种隐式
Scala 2.11 版库源代码中定义了超过 300 个隐式方法、隐式值和隐式类型。它们当中的大多数都是隐式方法,而隐式方法中的多数则被用于将某一类型转换成另一类型。掌握什么样的隐式可能会对代码产生影响对我们来说比较重要,因此我们会以小组的形式对这些隐式进行讨论,而不会是一一列举各个隐式。掌握每个隐式的详细内容绝非是最重要的,让读者能够“感觉”到隐式的类型才是最有益的。如果希望具备这一“直觉”,请概览本节。

我们之前已经讨论过作用于集合的 CanBuildFrom 构建器。现在有一个与它比较相似的CanCombineFrom 构建器,与其名字意思相符 CanCombineForm 构建器被许多操作用于组合实例。但是本节不会列出这些构造器的定义。

AnyVal 类型的所有伴生对象均提供了丰富的转换方法,例如:将 Int 值转换为 Long 值。大多数时候你只需要调用该类型的 toX 方法,如下所示:
```scala
object Int {
  ...
  implicit def int2long(x: Int): Long = x.toLong
  ...
}
```
Scala提供了一些 AnyVal 类型的隐式转换。需要注意的是由于 Scala 提供了隐式转换的功能,因此 Scala 无须像其他语言一样必须实现大多数常见的类型转换。

BigInt 和 BigDecimal 类定义在 scala.math 包中,它们可以转换来自 AnyVal 类型和 Java 中对应的实现类型。下列代码片段取自于 BigDecimal 伴生对象:
```scala
  /** Implicit conversion from `Int` to `BigDecimal`. */
  implicit def int2bigDecimal(i: Int): BigDecimal = apply(i)

  /** Implicit conversion from `Long` to `BigDecimal`. */
  implicit def long2bigDecimal(l: Long): BigDecimal = apply(l)

  /** Implicit conversion from `Double` to `BigDecimal`. */
  implicit def double2bigDecimal(d: Double): BigDecimal = decimal(d)

  /** Implicit conversion from `java.math.BigDecimal` to `scala.BigDecimal`. */
  implicit def javaBigDecimal2bigDecimal(x: BigDec): BigDecimal = apply(x)
```
调用 apply 方法时调用了 BigDecimal 伴生对象的 apply 工厂方法。这些隐式提供了另一种简便的方式调用这些方法。

Option 对象可以转换成包含 0 个或 1 个元素的列表:
```scala
implicit def option2Iterable[A](xo: Option[A]): Iterable[A] = xo.toList
```

Scala 使用了一些 Java 中的类型,像 Array[T] 和 String 类型。和它们对应的 ArrayOps[T] 和 StringOps 类型向所有的 Scala 集合提供常用的方法。因此,无论是转换成还是转换自这些封装类型的隐式转换都是非常有用的。其他的转换函数则定义在名称中包含 Wrapper 单词的类型中。

Predef 中定义了大多数的隐式定义。其中的一些隐式定义包含了 @inline 标注,这一标注鼓励编译器努力尝试将函数调用内联(inline),以减少栈帧开销。与之对应的是 @noinline 标注,该标注阻止编译器将方法调用内联化,即便是条件允许的情况。

一些方法能将某一类型转化为另一类型,例如:将某一类型封装成新的类型后,新的类型会提供新的方法:
```scala
@inline implicit def augmentString(x: String): StringOps = new StringOps(x)

@inline implicit def unaugmentString(x: StringOps): String = x.repr

implicit def tuple2ToZippedOps[T1, T2](x: (T1, T2))
  = new runtime.Tuple2Zipped.Ops(x)

implicit def tuple3ToZippedOps[T1, T2, T3](x: (T1, T2, T3))
  = new runtime.Tuple3Zipped.Ops(x)

implicit def genericArrayOps[T](xs: Array[T]): ArrayOps[T] = ...

implicit def booleanArrayOps(xs: Array[Boolean]): ArrayOps[Boolean] =
  = new ArrayOps.ofBoolean(xs)
...  // 与其他AnyVal类型相似的方法

implicit def refArrayOps[T <: AnyRef](xs: Array[T]): ArrayOps[T]
  = new ArrayOps.ofRef[T](xs)

@inline implicit def byteWrapper(x: Byte) = new runtime.RichByte(x)
...  // 与其他AnyVal类型相似的方法

implicit def genericWrapArray[T](xs: Array[T]): WrappedArray[T] = ...

implicit def wrapRefArray[T <: AnyRef](xs: Array[T]): WrappedArray[T] =...

implicit def wrapIntArray(xs: Array[Int]): WrappedArray[Int] = ...
...  // 与其他AnyVal类型相似的方法

implicit def wrapString(s: String): WrappedString = ...

implicit def unwrapString(ws: WrappedString): String = ...
```

如果你希望理解 runtime.Tuple2Zipped.Ops 方法的用意,首先你需要意识到大多数集合都提供了 zip 方法,该方法可以将两个集合连接起来。两个集合的元素组成一个新的元素的过程就好像合上拉链一样。
```scala
scala> val zipped = List(1,2,3) zip List(4,5,6)
zipped: List[(Int, Int)] = List((1,4), (2,5), (3,6))

//我们可以对合并后的集合执行成对的操作
scala> val products = zipped map { case (x,y) => x * y }
products: List[Int] = List(4, 10, 18)
```
请注意我们在输入参数为元组类型的匿名函数中应用了模式匹配,传递给匿名函数的一对Int 元素与 pair 元素相匹配。

Tuple2Zipper.Ops 和 Tuple3Zipper.Ops 提供了 invert 方法,该方法会将包含两个元素或三个元素的集合转换成包含二元元组或三元元组的集合。换言之,它们会压缩原本就定义在元组中的容器。例如:
```scala
scala> val pair = (List(1,2,3), List(4,5,6))
pair: (List[Int], List[Int]) = (List(1, 2, 3),List(4, 5, 6))

scala> val unpair = pair.invert
unpair: List[(Int, Int)] = List((1,4), (2,5), (3,6))

val pair = (List(1,2,3), List("one", "two", "three"))
tuple2ToZippedOps(pair) map {case (int, string) => (int*2, string.toUpperCase)}

val pair = (List(1,2,3), List(4,5,6))
pair map { case (int1, int2) => int1 + int2 }
```

在 Predef 对象中,还定义了许多转换成或转换自其他 Java 类型的转换方法,例如:
```scala
implicit def byte2Byte(x: Byte) = java.lang.Byte.valueOf(x)
implicit def Byte2byte(x: java.lang.Byte): Byte = x.byteValue
...  // Similar functions for the other AnyVal types.
```

下 面 的 代 码 将 java.util.Random 对 象 转 换 成 scala.util.Random 对 象:
```scala
implicit def javaRandomToRandom(r: java.util.Random): Random = new Random(r)
```

**scala.collection.convert 包中定义了一些 trait,这些 trait 提供了 Java 容器与 Scala 容器之间的转换方法。这为实现 Java 互操作性提供了便利。**事实上,出于性能方面的考虑,Scala 会尽可能避免执行类型转换。不过因为目标集合的抽象体是建立在底层容器的基础上,因此并不会造成太多的性能损耗。

DecorateAsJava 中定义了一些 Scala 容器,这些容器类是 Java 容器的装饰类(decoration)。为了便于读者阅读,我们对代码中返回的类型标注进行了换行处理,而下面代码中出现的 ju 和 jl 则分别对应了 DecorateAsJava 实际源码中的 java.lang 及 java.util 。各种名为 AsJava*的类型则是提供了这些转换操作的辅助类型:
```scala
implicit def asJavaIteratorConverter[A](i : Iterator[A]):
AsJava[ju.Iterator[A]] = ...
implicit def asJavaEnumerationConverter[A](i : Iterator[A]):
AsJavaEnumeration[A] = ...
implicit def asJavaIterableConverter[A](i : Iterable[A]):
AsJava[jl.Iterable[A]] = ...
implicit def asJavaCollectionConverter[A](i : Iterable[A]):
AsJavaCollection[A] = ...
implicit def bufferAsJavaListConverter[A](b : mutable.Buffer[A]):
AsJava[ju.List[A]] = ...
implicit def mutableSeqAsJavaListConverter[A](b : mutable.Seq[A]):
AsJava[ju.List[A]] = ...
implicit def seqAsJavaListConverter[A](b : Seq[A]):
AsJava[ju.List[A]] = ...
implicit def mutableSetAsJavaSetConverter[A](s : mutable.Set[A]):
AsJava[ju.Set[A]] = ...
implicit def setAsJavaSetConverter[A](s : Set[A]):
AsJava[ju.Set[A]] = ...
implicit def mutableMapAsJavaMapConverter[A, B](m : mutable.Map[A, B]):
AsJava[ju.Map[A, B]] = ...
implicit def asJavaDictionaryConverter[A, B](m : mutable.Map[A, B]):
AsJavaDictionary[A, B] = ...
implicit def mapAsJavaMapConverter[A, B](m : Map[A, B]):
AsJava[ju.Map[A, B]] = ...
implicit def mapAsJavaConcurrentMapConverter[A, B](m: concurrent.Map[A, B]):
AsJava[juc.ConcurrentMap[A, B]] = ...
```

我们可以使用 DecorateAsScala 中定义的隐式方法,将 Java 容器装饰成 Scala 容器:
```scala
implicit def asScalaIteratorConverter[A](i : ju.Iterator[A]):
AsScala[Iterator[A]] = ...
implicit def enumerationAsScalaIteratorConverter[A](i : ju.Enumeration[A]):
AsScala[Iterator[A]] = ...
implicit def iterableAsScalaIterableConverter[A](i : jl.Iterable[A]):
AsScala[Iterable[A]] = ...
implicit def collectionAsScalaIterableConverter[A](i : ju.Collection[A]):
AsScala[Iterable[A]] = ...
implicit def asScalaBufferConverter[A](l : ju.List[A]):
AsScala[mutable.Buffer[A]] = ...
implicit def asScalaSetConverter[A](s : ju.Set[A]):
AsScala[mutable.Set[A]] = ...
implicit def mapAsScalaMapConverter[A, B](m : ju.Map[A, B]):
AsScala[mutable.Map[A, B]] = ...
implicit def mapAsScalaConcurrentMapConverter[A, B](m: juc.ConcurrentMap[A, B]):
AsScala[concurrent.Map[A, B]] = ...
implicit def dictionaryAsScalaMapConverter[A, B](p: ju.Dictionary[A, B]):
AsScala[mutable.Map[A, B]] = ...
implicit def propertiesAsScalaMapConverter(p: ju.Properties):
AsScala[mutable.Map[String, String]] = ...
```

**尽管这些方法都是定义在特征中的,不过 JavaConverts 对象为你提供导入这些方法的快捷方式,你可以使用下列语句把这些方法加载到当前作用域中**:
```scala
import scala.collection.JavaConverters._
```
这些转换器的目的是让你能够对 Scala 容器调用 asJava 函数以生成对应的 Java 容器,对Java 容器调用 asScala 以生成 Java 容器。因此,这些方法有效地定义了 Scala 容器类型与Java 容器类型之间的 1 对 1 的关联关系。

但是更多时候,你希望能够选择输出容器的类型,而不是使用 asScala 或 asJava 方法提供的某些容器类型。这时, WrapAsJava 和 WrapAsScala 定义的一些额外的转换方法便能派得上用场。

下面列举了 WrapAsJava 中提供的一些方法:
```scala
implicit def asJavaIterator[A](it: Iterator[A]): ju.Iterator[A] = ...
implicit def asJavaEnumeration[A](it: Iterator[A]): ju.Enumeration[A] = ...
implicit def asJavaIterable[A](i: Iterable[A]): jl.Iterable[A] = ...
implicit def asJavaCollection[A](it: Iterable[A]): ju.Collection[A] = ...
implicit def bufferAsJavaList[A](b: mutable.Buffer[A]): ju.List[A] = ...
implicit def mutableSeqAsJavaList[A](seq: mutable.Seq[A]): ju.List[A] = ...
implicit def seqAsJavaList[A](seq: Seq[A]): ju.List[A] = ...
implicit def mutableSetAsJavaSet[A](s: mutable.Set[A]): ju.Set[A] = ...
implicit def setAsJavaSet[A](s: Set[A]): ju.Set[A] = ...
implicit def mutableMapAsJavaMap[A, B](m: mutable.Map[A, B]): ju.Map[A, B] =...
implicit def asJavaDictionary[A, B](m: mutable.Map[A, B]): ju.Dictionary[A, B]
implicit def mapAsJavaMap[A, B](m: Map[A, B]): ju.Map[A, B] = ...
implicit def mapAsJavaConcurrentMap[A, B](m: concurrent.Map[A, B]):
juc.ConcurrentMap[A, B] = ...
```
下面列出了 WrapAsScala 中定义的方法:
```scala
implicit def asScalaIterator[A](it: ju.Iterator[A]): Iterator[A] = ...
implicit def enumerationAsScalaIterator[A](i: ju.Enumeration[A]):
Iterator[A] = ...
implicit def iterableAsScalaIterable[A](i: jl.Iterable[A]): Iterable[A] = ...
implicit def collectionAsScalaIterable[A](i: ju.Collection[A]): Iterable[A]=...
implicit def asScalaBuffer[A](l: ju.List[A]): mutable.Buffer[A] = ...
implicit def asScalaSet[A](s: ju.Set[A]): mutable.Set[A] = ...
implicit def mapAsScalaMap[A, B](m: ju.Map[A, B]): mutable.Map[A, B] = ...
implicit def mapAsScalaConcurrentMap[A, B](m: juc.ConcurrentMap[A, B]):
concurrent.Map[A, B] = ...
implicit def dictionaryAsScalaMap[A, B](p: ju.Dictionary[A, B]):
mutable.Map[A, B] = ...
implicit def propertiesAsScalaMap(p: ju.Properties):
mutable.Map[String, String] = ...
[source,scala]
```

与 JavaConverters 相似,我们可以使用 JavaConversions 对象将定义在这些 trait 中的方法导入到当前作用域:
```scala
import scala.collection.JavaConversions._
```

对容器执行排序是非常常见的操作,因此 Scala 提供了一些 Ordering[T] 类型的隐式值,其 中 T 是 一 个 String 类 型 值, String 类 型 是 一 类 AnyVal 类 型, 可 以 转 换 成 Numeric 类型或用户自定义的顺序值(数值类型是对作用于数值的常见操作的一种抽象);请查看Ordered[T]中的定义。

我们不会列出一些在集合类型中定义的隐式 Ordering 值,不过在 Ordering[T] 类型中存在一些隐式定义:
```scala
implicit def ordered[A <% Comparable[A]]: Ordering[A] = ...
implicit def comparatorToOrdering[A](implicit c: Comparator[A]):Ordering[A]=...
implicit def seqDerivedOrdering[CC[X] <: scala.collection.Seq[X], T](
implicit ord: Ordering[T]): Ordering[CC[T]] = ...
implicit def infixOrderingOps[T](x: T)(implicit ord: Ordering[T]):
  Ordering[T]#Ops = ...
implicit def Option[T](implicit ord: Ordering[T]): Ordering[Option[T]] = ...
implicit def Iterable[T](implicit ord: Ordering[T]): Ordering[Iterable[T]] =...
implicit def Tuple2[T1, T2](implicit ord1: Ordering[T1], ord2: Ordering[T2]):
  Ordering[(T1, T2)] = ...
...  // 相似的函数:从Tuple3到Tuple9
```
最后,还有一些转换可用于构建“迷你- DSL 语言”的并发或管理进程。

首 先, scala.concurrent.duration 包中提供了一些用于定义持续时间的方法(我们会在第 17 章中看到这些类型以及用于调用并发的其他类型):
```scala
implicit def pairIntToDuration(p: (Int, TimeUnit)): Duration = ...
implicit def pairLongToDuration(p: (Long, TimeUnit)): FiniteDuration = ...
implicit def durationToPair(d: Duration): (Long, TimeUnit) = ...
```

下面列举了一些各种各样的转换方法,代码摘自 scala.concurrent 包的多个文件:
```scala
// scala.concurrent.FutureTaskRunner:
implicit def futureAsFunction[S](x: Future[S]): () => S

// scala.concurrent.JavaConversions:
implicit def asExecutionContext(exec: ExecutorService):
ExecutionContextExecutorService = ...
implicit def asExecutionContext(exec: Executor): ExecutionContextExecutor = ...

// scala.concurrent.Promise:
private implicit def internalExecutor: ExecutionContext = ...

// scala.concurrent.TaskRunner:
implicit def functionAsTask[S](fun: () => S): Task[S] = ...

// scala.concurrent.ThreadPoolRunner:
implicit def functionAsTask[S](fun: () => S): Task[S] = ...
implicit def futureAsFunction[S](x: Future[S]): () => S = ...
```

最后, Process 提供了一些与 UNIX shell 命令类似的方法,以支持操作系统进程操作:
```scala
implicit def buildersToProcess[T](builders: Seq[T])(
  implicit convert: T => Source): Seq[Source] = ...
implicit def builderToProcess(builder: JProcessBuilder): ProcessBuilder = ...
implicit def fileToProcess(file: File): FileBuilder = ...
implicit def urlToProcess(url: URL): URLBuilder = ...
implicit def stringToProcess(command: String): ProcessBuilder = ...
implicit def stringSeqToProcess(command: Seq[String]): ProcessBuilder = ...
```

这真是一个很长的代码!不过我希望你可以通过浏览这些代码对 Scala 库是如何支持并运用隐式能有一个大概的了解。

### 合理使用隐式
在构建 DSL 以及简化代码的 API 的过程中,隐式参数机制是一种非常强大的工具。不过由于传递的隐式参数以及隐式值几乎是不可见的,这就增加了理解代码的难度。所以,我们应该合理地使用隐式。

有一种可以提高隐式可见性的方法,即将隐式值统一放到名为 implicts 的特殊包或名为Implicits 的对象中。这种方式使得读者一旦遇到导入语句中的 implicit 字样时,便会留意到除了 Scala 内置的隐式之外,还存在一些其他的隐式。值得庆幸的是,目前有些 IDE也能够指出代码中存在的隐式。